# Fleming Pipeline - ipython notebook version

### Note on astrometry.net 

Shouldn't need a key to find RA/DEC coordinates since `force_upload=False`.

If the solve keeps timing out and you have `Config.astrometry_timeout` as a large value (or `-1`) then check the timeout in `~/.astroquery/config/astroquery.cfg`.


#### How to get a key
If you do want to upload (eg if machine takes too long solving) you will need a key and add it to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`.
The first is a config file (example layout is as below) latter is just a text file containing the key and nothing else.

[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Example config:
```
[astrometry_net]

api_key = XXXXX
timeout = 1200 ## 20 mins
server = http://npva.astrometry.net
```

In [1]:
## Imports
from pipeline import Reducer
from pipeline import ShiftFinder
from pipeline import FluxFinder
from pipeline import DataAnalyser
from pipeline import Constants
from pipeline import Utilities
from pipeline import Cataloguer

from datetime import datetime
import os
import importlib

In [2]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [7]:
config = Constants.Config(
    image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l138_0",
    bias_prefix = "bias_end",
    n_sets = 9,
    fits_extension = ".fits",
    fits_date_format = "%Y.%m.%dT%H:%M:%S.%f",
    has_filter_in_header = False,
)

In [8]:
r = Reducer.Reducer(config, "No filter") ## Only "No filter" for Trius

r.reduce(skip_existing=True)

[Reducer] Found 0 bias frames
[Reducer] Found 0 flatfield frames
[Reducer] Created master bias


/home/callum/envs/fleming/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/callum/envs/fleming/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: list index out of range

In [ ]:
c = Cataloguer.Cataloguer(config)

catalogue_image = os.path.join(config.image_dir, config.image_format_str.format(1, 1))

n_sources = c.generate_catalogue(catalogue_image, solve=True)
c.generate_image_times()

In [ ]:
sf = ShiftFinder.ShiftFinder(config, n_sources)

sf.generate_shifts()

In [ ]:
ff = FluxFinder.FluxFinder(config, n_sources)

ff.make_light_curves()

In [ ]:
da = DataAnalyser.DataAnalyser(config)

print("[Main] Creating average light curve")
da.create_avg_curve()
ff.plot_avg_light_curve(config.avg_curve_path, show=True)

print("[Main] Adjusting")
ff.create_adjusted_light_curves()

In [ ]:
print("[Main] Plotting variable light curves")
variable_ids = da.get_variables()
ff.plot_all_light_curves(variable_ids, adjusted=True, show=True)

In [ ]:
da.output_results()
da.create_thumbnails(ff)

## Debug land

Use at your own peril

In [1]:
## Imports
from pipeline import Reducer
from pipeline import ShiftFinder
from pipeline import FluxFinder
from pipeline import DataAnalyser
from pipeline import Constants
from pipeline import Utilities
from pipeline import Cataloguer

from datetime import datetime
import os
import importlib

In [5]:
Reducer = importlib.reload(Reducer)
ShiftFinder = importlib.reload(ShiftFinder)
FluxFinder = importlib.reload(FluxFinder)
DataAnalyser = importlib.reload(DataAnalyser)
Utilities = importlib.reload(Utilities)
Cataloguer = importlib.reload(Cataloguer)
Constants = importlib.reload(Constants)